# A Closer Look | PCA, SVD & Eigendecomposition
### Illustrating the connection between Principal Components Analysis, Singular Value Decomposition & Eigendecomposition

#### Objective

1. Walk through every step of PCA and show how it relates to singular value decomposition 

In [1]:
import numpy as np
from sklearn.datasets import make_classification
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

# generate random data
np.random.seed(5)

X, y = make_classification(n_samples=3, n_features=4)
print(X)

[[-0.68811868  1.26962382 -0.90187401 -1.43335751]
 [ 0.54329574  0.87531614 -1.47688597 -0.54723901]
 [-1.72325875  0.0507214   1.3888075  -0.79201964]]


#### Variance

$$
\begin{align}
Var(X) = & \frac{1}{N - 1}\sum_{i=1}^{N} (x_i - \bar{x})(x_i - \bar{x}) \\
= & \frac{1}{N - 1}\sum_{i=1}^{N} (x_i - \bar{x})^2
\end{align}
$$

#### Covariance

$$
Cov(x, y) = \frac{1}{N - 1}\sum_{i=1}^{N} (x_i - \bar{x})(y_i - \bar{y})
$$

Note that Covariance can be thought of as "variance with itself".

$$
\begin{align}
Cov(x, x) = & \frac{1}{N - 1}\sum_{i=1}^{N} (x_i - \bar{x})(x_i - \bar{x}) \\
Cov(x, x) = & \frac{1}{N - 1}\sum_{i=1}^{N} (x_i - \bar{x})^2 = Var(x)
\end{align}
$$

linear algebra notation for covariance:

$$
Cov(\mathbf{X}) = \frac{1}{N - 1}\mathbf{X}^T\mathbf{X}
$$

This results in a $p~x~p$ matrix where the $i^{th}$ diagonal element is the variance of the $i^{th}$ attribute of the original matrix, and all other elements $\left( x_{i,j}~where~i\neq j \right)$ are the covariances of the $i^{th}$ and $j^{th}$ attributes in the original matrix.

$$
Cov(\mathbf{X}) = \frac{1}{N - 1}\mathbf{X}^T\mathbf{X} = \begin{bmatrix}
                                                          \sigma_1^2 & \sigma_{1,2} & \sigma_{1,3} & \sigma_{1,4} \\
                                                          \sigma_{2,1} & \sigma_2^2 & \sigma_{2,3} & \sigma_{2,4} \\
                                                          \sigma_{3,1} & \sigma_{3,2} & \sigma_3^2 & \sigma_{3,4} \\
                                                          \sigma_{4,1} & \sigma_{4,2} & \sigma_{4,3} & \sigma_4^2 \\
                                                          \end{bmatrix}
$$

Also, note that $x_{i,j}$ of the covariance matrix will be the same as $x_{j,i}$, since: 

$$
\left( \frac{1}{N - 1}\sum_{i=1}^{N} (x_i - \bar{x})(y_i - \bar{y}) \right) = \left( \frac{1}{N - 1}\sum_{i=1}^{N} (y_i - \bar{y})(x_i - \bar{x}) \right)
$$

When one "centers" $\mathbf{X}$ (by subtracting the column-wise mean from each data point, leading to a new $\mathbf{X}$ where the column-wise means are equal to 0), the numerator of the covariance formula $\left( \sum_{i=1}^{N} (x_i - \bar{x})(y_i - \bar{y}) \right)$ becomes equivalent to the dot product (the sum of the element-wise products of two vectors).

$$
\begin{align}
Cov(x, y) = & \frac{1}{N - 1}\sum_{i=1}^{N} (x_i - \bar{x})(y_i - \bar{y}) \\
= & \frac{1}{N - 1}\sum_{i=1}^{N} (x_i - 0)(y_i - 0) \\
= & \frac{1}{N - 1}\sum_{i=1}^{N} (x_i)(y_i) \\
Numerator~of~covariance = & \sum_{i=1}^{N} x_iy_i = Dot~product\\
\end{align}
$$

In [8]:
# sklearn standardizing
scaler = StandardScaler()

print("\nSklearn implementation of standardizing.")
print("=========================================================")
X_transformed = scaler.fit_transform(X)
print(X_transformed)

# numpy standardizing
print("\nNumPy implementation of standardizing.")
print("=========================================================")
X_centered = (X - X.mean(axis=0)) / X.std(axis=0)
print(X_centered)


print("\n=========================================================")
print("=========================================================")

# numpy method for covariance matrix
print("\nNumPy implementation of calculating the covariance matrix.")
print("=========================================================")
cov_mat = np.cov(X_centered.T)
print(cov_mat)

# "hard-coded" version of calculating the covariance matrix
print("\nManually calculating the covariance matrix.")
print("=========================================================")
cov_mat1 = (1 / (X_centered.shape[0] - 1)) * (X_centered.T.dot(X_centered))
print(cov_mat1)


Sklearn implementation of standardizing.
[[-0.07061706  1.05886006 -0.46200914 -1.36269391]
 [ 1.25852557  0.28242682 -0.92654053  1.00891219]
 [-1.18790851 -1.34128687  1.38854967  0.35378171]]

NumPy implementation of standardizing.
[[-0.07061706  1.05886006 -0.46200914 -1.36269391]
 [ 1.25852557  0.28242682 -0.92654053  1.00891219]
 [-1.18790851 -1.34128687  1.38854967  0.35378171]]


NumPy implementation of calculating the covariance matrix.
[[ 1.5         0.93699694 -1.3914596   0.47285546]
 [ 0.93699694  1.5        -1.30666318 -0.81624053]
 [-1.3914596  -1.30666318  1.5         0.09301124]
 [ 0.47285546 -0.81624053  0.09301124  1.5       ]]

Manually calculating the covariance matrix.
[[ 1.5         0.93699694 -1.3914596   0.47285546]
 [ 0.93699694  1.5        -1.30666318 -0.81624053]
 [-1.3914596  -1.30666318  1.5         0.09301124]
 [ 0.47285546 -0.81624053  0.09301124  1.5       ]]


In [3]:
values, components = np.linalg.eig(cov_mat)
values1, components1 = np.linalg.eig(cov_mat1)


pca = PCA(svd_solver='full')
pca.fit(X_centered)
X_new = pca.transform(X_centered)

# why are first two the only matching ones??
print(pca.components_[:2,:])
print(components.T[:2,:])


[[-0.54143364 -0.56684159  0.61160043  0.10716878]
 [-0.40778653  0.33415932  0.0966321  -0.84422149]]
[[-0.54143364 -0.56684159  0.61160043  0.10716878]
 [ 0.40778653 -0.33415932 -0.0966321   0.84422149]]


In [4]:
# print(X_centered.shape)
# print(components.shape)
print(np.around(X_centered.dot(components)[:, :3], 2))
print(np.around(X_new, 2))

[[-0.99 -1.49  0.  ]
 [-1.3   1.36 -0.  ]
 [ 2.29  0.13  0.  ]]
[[-0.99  1.49 -0.  ]
 [-1.3  -1.36  0.  ]
 [ 2.29 -0.13 -0.  ]]


In [5]:


# sklearn transform
x_train_1 = scaler.transform(x_train)
x_test_1 = scaler.transform(x_test)

# Numpy transform
x_train_np = (x_train - x_train.mean(axis=0)) / x_train.std(axis=0)
x_test_np = (x_test - x_train.mean(axis=0)) / x_train.std(axis=0)

# sanity check ...
# print(x_train_1 == x_train_np)

# sklearn method
pca = PCA(svd_solver='full')
X_new = pca.fit_transform(X)
# X_new = pca.transform(X)
# print(np.around(pca.explained_variance_ratio_, 2))

# numpy method using cov = (1/N)*X^T*X...
cov = (1 / x_train_1.shape[0] - 1) * x_train_1.T.dot(x_train_1)
values, components = np.linalg.eig(cov)

# or just create the covariance matrix yourself...
cov_mat = np.cov(x_train_1.T)
values_1, components_1 = np.linalg.eig(cov_mat)

print("\n")
print("PCA via NumPy: Manual Covariance Matrix")
print("=========================================================")
print(np.around(components.T, 4))
print("\n")
print("PCA via Sklearn")
print("=========================================================")
print(np.around(pca.components_.T, 4))
print("\n")
print("PCA via NumPy: NumPy-Created Covariance Matrix")
print("=========================================================")
print(np.around(components_1.T, 4))

# SVD demo
X = np.array([[1,1,1,0,0],
              [3,3,3,0,0],
              [4,4,4,0,0],
              [5,5,5,0,0],
              [0,2,0,4,4],
              [0,0,0,5,5],
              [0,1,0,2,2]])

U, D, V = np.linalg.svd(X, full_matrices=False)


print("\n")
print("PCA via NumPy SVD")
print("=========================================================")
print(np.around(U.dot(np.diag(D)), 2))

print("\n")
print("PCA via Sklearn")
print("=========================================================")
print(np.around(X_new, 2))

# print(U)
# print(np.diag(D))
# print(V)

U = U[:, :3]
D = np.diag(D)[:3, :3]
V = V[:3,]

# print(np.around(U, 2))
# print(np.around(np.diag(D), 2))
# print(np.around(V, 2))

# svd reconstruction demo
u_new = U[:,:-1]
d_new = D[:2, :2]
v_new = V[:-1, :]

reconstructed = np.matmul(u_new.dot(d_new), v_new)

# little loss of data
# print(np.around(reconstructed, 2))
# print("\n")
# print(np.around(X, 2))


NameError: name 'x_train' is not defined